In [1]:
from unknown_data.test import TestHelper
from unknown_data import Config_db, Category

db_config = Config_db(
    dbms="postgresql",
    username="forensic_agent",
    password="0814",
    ip="13.124.25.47",
    port=5432,
    database_name="forensic_agent"
)

helper = TestHelper(db_config)

# helper.get_task_ids()

In [2]:
import datetime
import uuid

def category_make_test_artifacts(task_id: str, category: Category, artifact_list: list) -> int:
    data = helper.get_encoded_results(task_id, category)

    white_list = ["urls", "visits", "visited_links", "keywords", "keywork_search_terms", "autofill", "downloads", "download_url_chains", "logins"]
    white_list_set = set()
    for i in white_list:
        white_list_set.update(("chrome."+i, "edge."+i))

    cnt = 0
    for d in data.data:
        artifact_type = d.name.lower()

        if category == Category.BROWSER and artifact_type not in white_list_set:
            continue

        collected_at = datetime.datetime.now(tz=datetime.UTC)
        collected_at_str = collected_at.isoformat()
        
        # to_dict('records')를 사용하여 한 번에 모든 행을 딕셔너리 리스트로 변환 (훨씬 빠름!)
        records = d.data.to_dict('records')
        
        # 리스트 컴프리헨션으로 artifacts 생성 (for 루프보다 빠름)
        artifacts = [
            {
                "id": str(uuid.uuid4()),
                "artifact_type": artifact_type,
                "source": None,
                "data": record,
                "collected_at": collected_at_str
            }
            for record in records
        ]
        
        artifact_list.extend(artifacts)
        len_records = len(records)
        cnt += len_records
        # print(f"    {artifact_type}: {len_records}개 행 추가됨")
    del data
    return cnt

def make_test_artifacts(task_id: str) -> list:
    artifact_list = []

    for category in Category:
        # print(f"현재 {len(artifact_list)}개 아티팩트 존재.")
        cnt = category_make_test_artifacts(task_id, category, artifact_list)
        # print(f"{category.name}: {cnt}개 추가.")

    print(f"총 {len(artifact_list)}개의 artifact 생성됨\n")
    return artifact_list

In [3]:
task_ids = helper.get_task_ids()

=== 데이터베이스 Task 정보 ===
총 고유 Task 수: 8

Task ID: session-20251002-052932-151e52e9
  생성 시간 (최초): 2025-10-02 14:29:32 KST
  데이터 개수: 6
  모듈별 분포:
    - BROWSER_DATA: 1개
    - DELETED_FILES: 1개
    - LNK_DATA: 1개
    - MESSENGER_DATA: 1개
    - PREFETCH_DATA: 1개
    - USB_DATA: 1개

Task ID: session-20251002-051704-b6c68cbb
  생성 시간 (최초): 2025-10-02 14:17:04 KST
  데이터 개수: 6
  모듈별 분포:
    - BROWSER_DATA: 1개
    - DELETED_FILES: 1개
    - LNK_DATA: 1개
    - MESSENGER_DATA: 1개
    - PREFETCH_DATA: 1개
    - USB_DATA: 1개

Task ID: session-20251002-050523-6de09ba4
  생성 시간 (최초): 2025-10-02 14:05:23 KST
  데이터 개수: 5
  모듈별 분포:
    - BROWSER_DATA: 1개
    - LNK_DATA: 1개
    - MESSENGER_DATA: 1개
    - PREFETCH_DATA: 1개
    - USB_DATA: 1개

Task ID: session-20251002-045744-53bf2115
  생성 시간 (최초): 2025-10-02 13:57:44 KST
  데이터 개수: 6
  모듈별 분포:
    - BROWSER_DATA: 1개
    - DELETED_FILES: 1개
    - LNK_DATA: 1개
    - MESSENGER_DATA: 1개
    - PREFETCH_DATA: 1개
    - USB_DATA: 1개

Task ID: session-20251002-040520-28a9

In [8]:
for task_id in task_ids:
    if task_id == "session-20251002-050523-6de09ba4":
        continue
    print("task_id:", task_id)
    result = make_test_artifacts(task_id)
    print(f"total artifacts: {len(result)}")
    del result

task_id: session-20251002-052932-151e52e9
[2025-10-13 11:46:36] DataEncoder - Converted WebKit urls.last_visit_time to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit visits.visit_time to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit downloads.start_time to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit downloads.end_time to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit downloads.last_access_time to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit logins.date_created to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit logins.date_last_used to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit logins.date_received to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit logins.date_password_modified to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit insecure_credentials.create_time to datetime
[2025-10-13 11:46:36] DataEncoder - Converted WebKit cookies.creation_utc to

## 메모리 사용량 확인

In [5]:
import sys

# result = make_test_artifacts("session-20250930-060607-59984faf")
# result = make_test_artifacts("session-20251002-052932-151e52e9")
result = make_test_artifacts("session-20251002-045744-53bf2115")

# 메모리 사용량 확인
memory_usage = sys.getsizeof(result) / (1024 * 1024)  # MB 단위
print(f"artifact_list 메모리 사용량: {memory_usage:.2f} MB")
print(f"총 artifact 개수: {len(result):,}개")
print(f"평균 artifact 크기: {sys.getsizeof(result) / len(result) / 1024:.2f} KB")

[2025-10-13 11:44:57] DataEncoder - Converted WebKit urls.last_visit_time to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit visits.visit_time to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit downloads.start_time to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit downloads.end_time to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit downloads.last_access_time to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit logins.date_created to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit logins.date_last_used to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit logins.date_received to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit logins.date_password_modified to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit logins.date_last_filled to datetime
[2025-10-13 11:44:57] DataEncoder - Converted WebKit insecure_credentials.create_time to datetime
[2025-10-13 11:44:57] DataEnc

In [6]:
import datetime
def date_from_webkit(webkit_timestamp):
    epoch_start = datetime.datetime(1601,1,1)
    delta = datetime.timedelta(microseconds=int(webkit_timestamp))
    return epoch_start + delta

def check_date_from_now(webkit_timestamp, month: int) -> bool:
    """Webkit timestamp가 현재로부터 month개월 이내인지 확인"""
    if not isinstance(webkit_timestamp, (int, float)):
        return False
    if webkit_timestamp <= 0:
        return False
    
    # # ✅ Webkit 타임스탬프 범위 검증 (1601년 ~ 3000년 정도)
    # if webkit_timestamp < 0 or webkit_timestamp > 50000000000000000:  # 약 3000년
    #     return False
    
    try:
        data_time = date_from_webkit(webkit_timestamp)
        target_time = datetime.datetime.now() - datetime.timedelta(weeks=month * 4)
        return target_time < data_time
    except (ValueError, OverflowError):
        print("!!!")
        return False

def check_timestamp_from_now(timestamp, month: int) -> bool:
    """Unix timestamp가 현재로부터 month개월 이내인지 확인"""
    if not isinstance(timestamp, (int, float)):
        return False
    if timestamp <= 0:
        return False
    
    try:
        data_time = datetime.datetime.fromtimestamp(timestamp)
        target_time = datetime.datetime.now() - datetime.timedelta(weeks=month * 4)
        return target_time < data_time
    except (ValueError, OSError):  # 잘못된 timestamp 값
        return False


stat = {}

now_type = ""
for i, artifact in enumerate(result, 1):
    if now_type != artifact["artifact_type"]:
        now_type = str(artifact["artifact_type"])
        if now_type == "search_directories":
            continue
        print(now_type)
        stat.update({now_type: [0, 0]})
        keys = list(artifact["data"].keys())
        # keys = [key for key in keys if isinstance(key, str) and key.find("date") >= 0 or key.find("time") >= 0]
        for key in keys:
            print(key, artifact["data"][key], end=",")
        
        timestamp = None
        timestamps = [key for key in keys if key.find("timestamp") >= 0]
        if timestamps:
            timestamp = timestamps[0]
        print()

    if not keys:
        continue

    if now_type == "prefetch_files":
        stat[now_type][0] += 1
        try:
            last_run_time = datetime.datetime.strptime(
                artifact["data"][keys[0]], 
                "%Y-%m-%dT%H:%M:%SZ"
            )
            target_time = datetime.datetime.now() - datetime.timedelta(weeks=12)  # 3개월 = 12주
            if target_time < last_run_time:
                stat[now_type][1] += 1
        except (ValueError, KeyError, TypeError):
            print("!")
            pass  # 파싱 실패 시 스킵
        continue

    if now_type == "usb_devices":
        stat[now_type][0] += 1
        continue
    
    if now_type.find("chrome") < 0 and now_type.find("edge") < 0:
        if not timestamp:
            continue

        time = artifact["data"].get(timestamp)
        if not time:
            continue

        stat[now_type][0] += 1
        if check_timestamp_from_now(time, 6):
            stat[now_type][1] += 1
        continue

    try:
        time_value = int(artifact["data"][keys[-1]])
        stat[now_type][0] += 1
        if check_date_from_now(time_value, 3):
            stat[now_type][1] += 1
    except (ValueError, KeyError, TypeError):
        pass  # 변환 실패 시 스킵

for k, v in stat.items():
    print(str(k).ljust(20), end=" ")
    print(f"total data: {str(v[0]).ljust(5)}, in-time data: {v[1]}")

chrome.urls
id 53,url https://colab.research.google.com/notebooks/welcome.ipynb#scrollTo=lSrWNr3MuFUS,title Colab 시작하기 - Colab,hidden 0,typed_count 0,visit_count 6,last_visit_time 2025-09-24 07:43:47+00:00,
chrome.visits
id 53449,url 26316,app_id ,from_visit 0,segment_id 0,transition 805306368,visit_time 2025-07-04 05:47:50+00:00,opener_visit 53450,visit_duration 0,visited_link_id 0,is_known_to_sync 1,originator_visit_id 21978,external_referrer_url https://www.naver.com/,originator_cache_guid suZG25Nz7o6aW3duZPhpMA==,originator_from_visit 0,originator_opener_visit 21977,consider_for_ntp_most_visited 1,incremented_omnibox_typed_score 0,
chrome.downloads
id 1,etag "67cb13bb-d127cdf4",guid cbcb0d3f-7ae8-4415-a1ac-2f3934acab80,hash ,state 1,opened 1,tab_url https://www.kali.org/get-kali/#kali-virtual-machines,end_time 2025-04-02 00:20:24+00:00,referrer https://www.kali.org/,site_url ,by_ext_id ,mime_type application/x-compressed,transient 0,start_time 2025-04-02 00:17:45+00:00,by_ext_name 

## 메모리 최적화 버전 (제너레이터 사용)
대량 데이터를 한 번에 메모리에 올리지 않고 필요할 때마다 생성

In [7]:
def generate_artifacts(task_id: str, batch_size: int = 1000):
    """
    제너레이터 방식: 메모리 효율적으로 artifacts를 배치 단위로 생성
    
    Args:
        task_id: 작업 ID
        batch_size: 한 번에 반환할 artifacts 개수
        
    Yields:
        batch_size 개의 artifacts 리스트
    """
    white_list = ["urls", "visits", "visited_links", "keywords", "keywork_search_terms", 
                  "autofill", "downloads", "download_url_chains", "logins"]
    white_list_set = set()
    for i in white_list:
        white_list_set.update(("chrome."+i, "edge."+i))
    
    for category in Category:
        data = helper.get_encoded_results(task_id, category)
        
        for d in data.data:
            artifact_type = d.name.lower()
            
            if category == Category.BROWSER and artifact_type not in white_list_set:
                continue
            
            collected_at_str = datetime.datetime.now(tz=datetime.UTC).isoformat()
            
            # 대용량 데이터를 청크 단위로 처리
            total_rows = len(d.data)
            for start_idx in range(0, min(total_rows, 10000), batch_size):
                end_idx = min(start_idx + batch_size, total_rows, 10000)
                
                # 청크만 메모리에 로드
                chunk_records = d.data.iloc[start_idx:end_idx].to_dict('records')
                
                artifacts = [
                    {
                        "id": str(uuid.uuid4()),
                        "artifact_type": artifact_type,
                        "source": None,
                        "data": record,
                        "collected_at": collected_at_str
                    }
                    for record in chunk_records
                ]
                
                yield artifacts
                
        del data  # 메모리 해제


# 사용 예시 1: 배치 처리
def process_in_batches(task_id: str):
    """배치 단위로 처리 (예: DB에 저장)"""
    total_count = 0
    
    for batch in generate_artifacts(task_id, batch_size=1000):
        # 여기서 배치 단위로 DB 저장 또는 다른 처리
        total_count += len(batch)
        print(f"진행 중... 현재까지 {total_count:,}개 처리")
        
    print(f"\n총 {total_count:,}개 처리 완료")
    return total_count


# 사용 예시 2: 필요하면 전체 리스트로 변환
def make_test_artifacts_optimized(task_id: str) -> list:
    """메모리가 충분하면 전체를 리스트로 변환"""
    artifact_list = []
    
    for batch in generate_artifacts(task_id, batch_size=5000):
        artifact_list.extend(batch)
    
    print(f"\n총 {len(artifact_list):,}개의 artifact 생성됨")
    return artifact_list

## 권장사항 정리

### 5만 개 데이터 처리 시:

**1. 메모리가 충분한 경우 (8GB+ RAM)**
- 현재 구현 그대로 사용 OK
- 5만 개 × 평균 1-2KB = 약 50-100MB 정도 (문제없음)

**2. DB에 바로 저장하는 경우**
- 제너레이터 방식 사용 (`process_in_batches`)
- 1000개씩 배치로 INSERT → 메모리 사용량 최소화

**3. 데이터가 더 커질 가능성이 있는 경우**
- 제너레이터 방식으로 미리 구현
- 확장성 확보

### 추가 최적화 팁:
- `data` 필드의 불필요한 컬럼 제거
- JSON 직렬화 전 데이터 타입 최적화 (datetime → string 등)
- `del` 키워드로 사용 후 명시적 메모리 해제 (현재 코드에 이미 있음 ✅)